# Анализ базы данных сервиса для чтения книг

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

---

В ходе проекта анализируется база данных сервиса для чтения книг с целью формирования рекомендаций по составлению предложения для клиентов.

Массивы данных содержат в себе информацию о книгах (название, количество страниц, издатель, год публикации), авторах, издателях, пользовательских оценках и обзорах на книги. 

В процессе исследования решается ряд задач:
- обзор таблиц; 
- составление запросов к базе данных для ответа на вопросы:
    + сколько книг вышло после 1 января 2000 года;
    + количество обзоров и средняя оценка каждой книги;
    + определение издательства, выпустившего наибольшее число книг толще 50 страниц;
    + определение автора с самой высокой средней оценкой книг (для книг с 50 и более оценками);
    + подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок.
---


Импорт библиотек и базы данных:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': '-----', # имя пользователя
 'pwd': '-----', # пароль
 'host': '-----',
 'port': 1111, # порт подключения
 'db': '-----'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 


# Описание данных

Выведем все таблицы на экран:

**books** - книги

In [2]:
query_books = '''
                SELECT * FROM books
                LIMIT 5
              '''
pd.io.sql.read_sql(query_books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query_books_n = '''
                SELECT COUNT(*) FROM books
              '''
pd.io.sql.read_sql(query_books_n, con = engine) 

,count
0,1000


В датафрейме **books** представлена информация о книгах. В нем 6 столбцов: 

- **book_id** - идентификатор книги;
- **author_id** - идентификатор автора;
- **title** - название;
- **num_pages** - количество страниц;
- **publication_date** - дата издания;
- **publisher_id** - идентификатор издателя;

Общее количество записей - 1000.

---

**authors** - авторы

In [4]:
query_authors = '''
                SELECT * FROM authors
                LIMIT 5
              '''
pd.io.sql.read_sql(query_authors, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query_authors_n = '''
                SELECT COUNT(*) FROM authors
              '''
pd.io.sql.read_sql(query_authors_n, con = engine) 

,count
0,636


В датафрейме **authors** представлена информация об авторах книг. В нем всего два столбца: 

- **author_id** - идентификатор автора;
- **author** - автор.

Общее количество записей - 636.

---

**publishers** - издатели

In [6]:
query_publishers = '''
                SELECT * FROM publishers
                LIMIT 5
              '''
pd.io.sql.read_sql(query_publishers, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query_publishers_n = '''
                SELECT COUNT(*) FROM publishers
              '''
pd.io.sql.read_sql(query_publishers_n, con = engine) 

,count
0,340


В датафрейме **authors** представлена информация об издателях книг. В нем тоже два столбца: 

- **publisher_id** - идентификатор издателя;
- **publisher** - издатель.

Общее количество записей - 340.

---

**ratings** - пользовательские оценки книг

In [8]:
query_ratings = '''
                SELECT * FROM ratings
                LIMIT 5
              '''
pd.io.sql.read_sql(query_ratings, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
query_ratings_n = '''
                SELECT COUNT(*) FROM ratings
              '''
pd.io.sql.read_sql(query_ratings_n, con = engine) 

,count
0,6456


В датафрейме **ratings** представлена информация об оценках книг пользователями. В нем 4 столбца: 

- **rating_id** - идентификатор оценки;
- **book_id** - идентификатор книги;
- **username** - имя пользователя, оставившего оценку;
- **rating** - оценка книги.

Общее количество записей - 6456.

---

**reviews** - пользовательские обзоры на книги

In [10]:
query_reviews = '''
                SELECT * FROM reviews
                LIMIT 5
              '''
pd.io.sql.read_sql(query_reviews, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [11]:
query_reviews_n = '''
                SELECT COUNT(*) FROM reviews
              '''
pd.io.sql.read_sql(query_reviews_n, con = engine) 

,count
0,2793


В датафрейме **reviews** представлена информация об отзывах пользователей на книги. В нем 4 столбца: 

- **review_id** - идентификатор обзора;
- **book_id** - идентификатор книги;
- **username** - имя пользователя, написавшего обзор;
- **text** - текст обзора.

Общее количество записей - 2793.

---

# Составление запросов к базе данных

**1. Количество книг, вышедших после 1 января 2000 года**

In [12]:
query1 = '''
            SELECT 
                COUNT(book_id)
            FROM books
            WHERE
                CAST(publication_date AS DATE) > '2000-01-01';
        '''
pd.io.sql.read_sql(query1, con = engine) 

,count
0,819


В сервисе для чтения находится 819 книг, изданных после 01.01.2000 г. 

**2. Количество обзоров и средняя оценка каждой книги**

In [13]:
query2 = '''
            SELECT
                books.book_id, 
                books.title,
                COUNT(DISTINCT review_id) as num_reviews,
                AVG(rating) as average_rating
            FROM books
            INNER JOIN reviews ON books.book_id = reviews.book_id    
            INNER JOIN ratings ON books.book_id = ratings.book_id
            GROUP BY 
                books.book_id
            LIMIT 10;
        '''
pd.io.sql.read_sql(query2, con = engine)

,book_id,title,num_reviews,average_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
5,6,1st to Die (Women's Murder Club #1),4,3.500000
6,7,2nd Chance (Women's Murder Club #2),3,3.000000
7,8,4th of July (Women's Murder Club #4),3,4.000000
8,9,A Beautiful Mind,2,4.250000
9,10,A Bend in the Road,3,3.200000


Для каждой книги выведено количество обзоров **num_reviews** и средняя оценка **average_rating**. Для удобства восприятия число отображаемых строк ограничено первыми десятью.

**3. Определение издательства, выпустившего наибольшее число книг толще 50 страниц**

In [14]:
query3 = '''
           
            SELECT 
                publisher, 
                COUNT(books.num_pages > 50) AS num_non_booklets
            FROM books
            INNER JOIN publishers ON books.publisher_id = publishers.publisher_id   
            GROUP BY publisher 
            HAVING COUNT(publisher) = 
                (SELECT
                    MAX(subquery.num_non_booklets)
                FROM
                    (SELECT
                        publisher,
                        COUNT(books.num_pages > 50) AS num_non_booklets
                    FROM books
                    INNER JOIN publishers ON books.publisher_id = publishers.publisher_id    
                    GROUP BY 
                        publisher) AS subquery);
                  
        '''
pd.io.sql.read_sql(query3, con = engine)

,publisher,num_non_booklets
0,Penguin Books,42


Наибольшее число книг с числом страниц более 50 - 42 книги - выпустило издательство Penguin Books.

**4. Определение автора с самой высокой средней оценкой книг (для книг с 50 и более оценками)**

In [15]:
query4 = '''
            SELECT 
                author,
                AVG(rating) AS avg_rating
            FROM books   
            INNER JOIN ratings ON books.book_id = ratings.book_id
            INNER JOIN authors ON books.author_id = authors.author_id
            WHERE books.book_id IN
                (SELECT
                    book_id
                FROM ratings
                GROUP BY book_id
                HAVING COUNT(rating) >= 50)
            GROUP BY author
            HAVING AVG(rating) = 
                (SELECT
                    MAX(subquery.avg_rating)
                FROM
                    (SELECT
                        author,
                        AVG(rating) AS avg_rating
                    FROM books   
                    INNER JOIN ratings ON books.book_id = ratings.book_id
                    INNER JOIN authors ON books.author_id = authors.author_id
                    WHERE books.book_id IN
                        (SELECT
                            book_id
                        FROM ratings
                        GROUP BY book_id
                        HAVING COUNT(rating) >= 50)
                    GROUP BY author
                    ) AS subquery
                );  
        '''
pd.io.sql.read_sql(query4, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг (для книг с 50 и более оценками) - Дж.К.Роулинг (с иллюстратором М.Гранпрэ).

**5. Подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок**

In [16]:
query5 = '''
            SELECT
                AVG(subquery.num_reviews) AS avg_num_reviews
            FROM
                (SELECT
                    username, 
                    COUNT(username) as num_reviews
                FROM reviews
                WHERE username IN 
                    (SELECT
                        username
                     FROM ratings
                     GROUP BY username
                     HAVING COUNT(rating_id) > 50
                    )
                GROUP BY username
                ) AS subquery;

        '''
pd.io.sql.read_sql(query5, con = engine)   

,avg_num_reviews
0,24.333333


В среднем пользователи, поставившие более 50 оценок, написали по 24,33 обзора.

# Вывод

В ходе проекта получена следующая информация:
- в сервисе для чтения зарегистрировано 1000 книг, из них 819 издано после 01.01.2000 г.
- для каждой книги выведено количество обзоров и средняя оценка пользователей;
- издательство Penguin Books выпустило наибольшее количество книг (42) с числом страниц более 50; 
- автор с самой высокой средней оценкой книг (для книг с 50 и более оценками) - Дж.К.Роулинг (с иллюстрациями М.Гранпрэ);
- пользователи, поставившие более 50 оценок, написали в среднем по 24,33 обзора.